# Trabajo Final - Redes Neuronales

## Importo las librerias necesarias


In [ ]:
from autoencoder import *
from clasificador import *

## Red neuronal autoencoder convolucional de varias capas

### Importo data set

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

# Download and load the training data
train_set_orig = datasets.FashionMNIST(
    'MNIST_data/', download=True, train=True,  transform=transform)
valid_set_orig = datasets.FashionMNIST(
    'MNIST_data/', download=True, train=False, transform=transform)

In [ ]:
figure = plt.figure()
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    j = torch.randint(len(train_set_orig), size=(1,)).item()
    image, label = train_set_orig[j]
    figure.add_subplot(rows, cols, i)
    plt.title(train_set_orig.classes[label])
    plt.axis("off")
    plt.imshow(image.squeeze(), cmap="gray")
plt.show()

### Creo data set personalizado

In [ ]:
train_set = CustomDataset(train_set_orig)
valid_set = CustomDataset(valid_set_orig)

### Parte 1: red autoencoder

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, n, p=0.2):
        super(Autoencoder, self).__init__()
        self.flatten = nn.Flatten()
        self.encoder = nn.Sequential(
            # Convolucional 1
            nn.Conv2d(1, 16, kernel_size=3),  # 1x28x28 -> 16x26x26
            nn.ReLU(),
            nn.Dropout(p),
            nn.MaxPool2d(2,2), # 16x26x26 -> 16x13x13
            # Convolucional 2
            nn.Conv2d(16, 32, kernel_size=3),  # 16x13x13 -> 32x11x11
            nn.ReLU(),
            nn.Dropout(p),
            nn.MaxPool2d(2, 2),  # 32x11x11 -> 32x5x5
            # Linear
            nn.Flatten(),  # 32x5x5 -> 32*5*5
            nn.Linear(32*5*5, n),  # fully connected 32*5*5 -> n
            nn.ReLU(),
            nn.Dropout(p)
        )
        self.decoder = nn.Sequential(
            # Linear
            nn.Linear(n, 32*5*5),  # fully connected n -> 32*5*5
            nn.ReLU(),
            nn.Dropout(p),
            nn.Unflatten(1, (32,5,5)),  # 32*5*5 -> 32x5x5
            # Convolucional transpose (de la segunda convolucional)
            nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, output_padding=1),  # 32x5x5 -> 16x13x13
            nn.ReLU(),
            nn.Dropout(p),
            # Convolucional transpose (de la primera convolucional)
            nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, output_padding=1),  # 16x13x13 -> 1x28x28
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

## Modelo sin entrenar

In [ ]:
# 2.6)
n = 64
p = 0.2
autoencoder_conv = Autoencoder(n, p)
model = autoencoder_conv

In [ ]:
autoencoder_conv.encoder

In [ ]:
autoencoder_conv.decoder

In [ ]:
def batch(x):
    return x.unsqueeze(0)  # 28x28 -> 1x28x28


def unbatch(x):
    return x.squeeze().detach().cpu().numpy()  # 1x28x28 -> 28x28

In [ ]:
image, _ = train_set[0]
pred = model(batch(image))
print(image.size())
print(unbatch(pred).shape)
print(pred.size())
print(unbatch(pred).shape)

In [ ]:
# miramos que imagene predice el modelo sin entrenar
figure = plt.figure()
rows, cols = 3, 2
i = 0
for row in range(1, rows + 1):
    j = torch.randint(len(train_set), size=(1,)).item()
    i += 1
    image, _ = train_set[j]
    figure.add_subplot(rows, cols, i)
    if row == 1:
        plt.title('Original')
    plt.axis("off")
    plt.imshow(image.squeeze(), cmap="gray")
    i += 1
    figure.add_subplot(rows, cols, i)
    if row == 1:
        plt.title('Reconstructed')
    plt.axis("off")
    image_pred = unbatch(model(batch(image)))
    plt.imshow(image_pred.squeeze(), cmap="gray")
plt.show()

## Parte 2: entrenenado el autoencoder

### Modelo Base

In [ ]:
indices = torch.randint(len(valid_set), size=(3,))
indices

In [ ]:
model_orig = model_generator(
    Autoencoder, 64, 0.2, 30, 100, 'Adam', 1e-3, train_set, valid_set)

In [ ]:
test_model(model_orig, valid_set, indices)

## Variar algunos parametros

### vario dropout

In [ ]:
model_drop05 = model_generator(
    Autoencoder, 64, 0.5, 30, 100, 'Adam', 1e-3, train_set, valid_set)

In [ ]:
test_model(model_drop05, valid_set)

In [ ]:
model_64_drop01 = model_generator(
    Autoencoder, 64, 0.1, 30, 100, 'Adam', 1e-3, train_set, valid_set)

In [ ]:
test_model(model_64_drop01, valid_set, indices)

In [ ]:
model_256_drop01 = model_generator(
    Autoencoder, 256, 0.1, 30, 100, 'Adam', 1e-3, train_set, valid_set)

In [ ]:
test_model(model_256_drop01, valid_set, indices)

### Variando optimizador

In [ ]:
model_drop01_sgd = model_generator(
    Autoencoder, 64, 0.2, 30, 100, 'SGD', 1e-3, train_set, valid_set)

In [ ]:
test_model(model_drop01_sgd, valid_set, indices)

## Parte 3: Definiendo un clasificador convolucional reutilizando el encoder

In [ ]:
import copy


class Classifier_Conv(nn.Module):
    def __init__(self, n, encoder, p=0.2):
        super(Classifier_Conv, self).__init__()
        self.flatten = nn.Flatten()
        self.encoder = copy.deepcopy(encoder)
        self.classifier = nn.Sequential(
            # Linear
            nn.Linear(n, 10),  # fully connected n -> 32*5*5
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)
        return x

### clasificador usando el modelo original

In [ ]:
model_classifier_orig = classifier_generator(
    Classifier_Conv, 64, model_orig, 0.2, 30, 100, 'Adam', 1e-3, train_set_orig, valid_set_orig)

## Parte 4:

## Entrenando solo la capa clasificadora

In [ ]:
classifier_6 = classifier_generator(Classifier_Conv, 64, model_orig,
                                    0.2, 30, 100, 'Adam classifier', 1e-3, train_set_orig, valid_set_orig)